In [127]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
from astropy import cosmology
import matplotlib.pyplot as plt
import getdist as gd
from getdist import MCSamples
import getdist.plots as gdplot
import matplotlib
from scipy.stats import norm
import matplotlib.mlab as mlab
from autogalaxy.util import cosmology_util
from astropy import constants
import re

cosmo = cosmology.Planck15
fig_path = '/Users/dgmt59/Documents/Plots/weak_lensing/'

In [128]:
from autoconf import conf
import autofit as af
import autolens as al
import autolens.plot as aplt
from astropy.io import fits

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
from matplotlib import gridspec as gridspec
from matplotlib.colors import SymLogNorm
import math


In [129]:
#import autogalaxy.util.convert as convert


%matplotlib inline

workspace_path = "/Users/dgmt59/PycharmProjects/autolens_slacs/"
output_path = "/Users/dgmt59/output"
aggregator_results_path_1 = output_path + "/slacs_shu_bspline_clean/slacs1430+4105"

fits_header_path = "/Users/dgmt59/Documents/Data/SLACS_headers/SLACSJ1430+4105_F814W_4_biz.fits"
shu_data_path = "/Users/dgmt59/PycharmProjects/autolens_slacs_pre_v_1/dataset/slacs_shu/slacs1430+4105/F814W_image.fits"

agg = af.Aggregator(directory=str(aggregator_results_path_1), completed_only=True)

Aggregator loading phases... could take some time.

 A total of 9 phases and results were found.


In [130]:
from astropy.io import fits
data_path = "/Users/dgmt59/Documents/Data/slacs_weak_lensing/1430+4105_F814W_drz_sci_clean.shears"
field_image_path = "/Users/dgmt59/Documents/Data/slacs_weak_lensing/1430+4105_F814W_drz_sci.fits"

weak_shear = fits.open(data_path)

image_shu = fits.open(fits_header_path)[0]
image_shu_transform= fits.open(shu_data_path)[0]


data = weak_shear[1].data

field_image_data  = get_pkg_data_filename(field_image_path)

field_image = fits.getdata(field_image_data, ext=0)

In [131]:
print(weak_shear[1].columns.names)

['x', 'y', 'xx', 'yy', 'xy', 'xxxx', 'yyyy', 'xxyy', 'xyyy', 'xxxy', 'prob', 'skymed', 'skysd', 'skysw', 'exp_time', 'radius', 'x_err', 'y_err', 'xx_err', 'yy_err', 'xy_err', 'xxxx_err', 'yyyy_err', 'xxyy_err', 'xyyy_err', 'xxxy_err', 'prob_err', 'skymed_err', 'skysd_err', 'skysw_err', 'exp_time_err', 'ra', 'dec', 'e1', 'e2', 'e1_err', 'e2_err', 'ell', 'pa', 'gal_size', 'a', 'b', 'NUMBER', 'X_IMAGE', 'Y_IMAGE', 'X_WORLD', 'Y_WORLD', 'THETA_WORLD', 'THETA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'FWHM_IMAGE', 'MAG_ISO', 'MU_MAX', 'MAG_AUTO', 'MAGERR_AUTO', 'MAGERR_ISO', 'ISOAREA_IMAGE', 'CLASS_STAR', 'BACKGROUND', 'FLAGS', 'FLUX_AUTO', 'FLUXERR_AUTO', 'KRON_RADIUS', 'RA', 'DEC', 'galStarFlag', 'j9op36bf_X_IMAGE', 'j9op36bf_Y_IMAGE', 'j9op36bg_X_IMAGE', 'j9op36bg_Y_IMAGE', 'j9op36bi_X_IMAGE', 'j9op36bi_Y_IMAGE', 'j9op36bk_X_IMAGE', 'j9op36bk_Y_IMAGE', 'shear', 'nExposures', 'xx_uncorrected', 'yy_uncorrected', 'gamma1', 'gamma2', 'clean']


In [175]:
import numpy as np
from astropy.io import fits

import ipdb as pdb

def calc_shear_err(shear_catalogue,
                   outfile,
                   min_rad=6,
                   catalogue=None):
    
    if catalogue is None:
        catalogue = fits.open(shear_catalogue)[1].data

    weight = catalogue['radius']
    weight[catalogue['radius'] < min_rad] = min_rad
    
    beta = 1./(2.*catalogue['gal_size']**2*(catalogue['shear']**2+weight**2))

    u1 = beta*(-catalogue['xxxx']+catalogue['yyyy']) 
    u2 = -2.*beta*(catalogue['xxxy']+catalogue['xyyy'])
    u1_err = np.sqrt(np.square(catalogue['xxxx_err'])+np.square(catalogue['yyyy_err']))
    u2_err = np.sqrt(np.square(catalogue['xxxy_err'])+np.square(catalogue['xyyy_err']))

    gal_lambda=beta*(catalogue['xxxx']+2.*catalogue['xxyy']+catalogue['yyyy'])
  
    
    ellipticity_sqr = catalogue['e1']**2+catalogue['e2']**2
    e_dot_u = catalogue['e1']*u1+catalogue['e2']*u2

    e1_err = catalogue['e1_err']
    e2_err = catalogue['e2_err']

    G = 2-np.nanmedian(ellipticity_sqr)-\
            0.5*np.nanmedian(gal_lambda)-\
            0.5*np.nanmedian(e_dot_u)

    G_err = G*np.sqrt(np.square(-2*catalogue['e1'])*np.square(e1_err)+\
                      np.square(-0.5*u2)*np.square(e1_err)+\
                        np.square(2*catalogue['e2'])**np.square(e2_err)+\
                        np.square(0.5*u1)*np.square(e2_err)+\
                        np.square(0.5*catalogue['e1']*u1_err)+\
                        np.square(0.5*catalogue['e2']*u2_err)+\
                        np.square(-0.5*beta)*(
                                np.square(catalogue['xxxx_err'])+
                                np.square(catalogue['xxyy_err'])+
                                np.square(catalogue['yyyy_err'])))

    gamma1=catalogue['e1']/G
    gamma2=catalogue['e2']/G

    gamma1_err = gamma1*np.sqrt(np.square(e1_err/catalogue['e1'])+\
                                 np.square(G_err/G))
    gamma2_err = gamma2*np.sqrt(np.square(e2_err/catalogue['e2'])+\
                                 np.square(G_err/G))

    fits_cols = []
    for iName in catalogue.columns.names:
        fits_cols.append( fits.Column(name=iName, format=catalogue[iName].dtype, array=catalogue[iName] ) )
        
    newcol = [ fits.Column(name='gamma1_err', format=gamma1_err.dtype, array=gamma1_err),
               fits.Column(name='gamma2_err', format=gamma2_err.dtype, array=gamma2_err) ]

 
    hdu = fits.BinTableHDU.from_columns(fits_cols + newcol)
    hdu.writeto(outfile, overwrite=True,output_verify='ignore')

In [176]:
calc_shear_err(shear_catalogue=data_path, outfile=f"{data_path}_errors")

In [177]:
data_err = fits.open(f"{data_path}_errors")[1].data


print(np.mean(data_err['gamma1_err']))
print(np.mean(data_err['gamma2_err']))

-0.006392441395311924
-0.018866055605301715


In [147]:
output_path = "/Users/dgmt59/output"
aggregator_results_path = output_path + "/slacs_shu_bspline_clean/"
weak_data_path = "/Users/dgmt59/Documents/Data/slacs_weak_lensing/"
header_path = "/Users/dgmt59/Documents/Data/SLACS_headers/"
slacs_shu_path = "/Users/dgmt59/PycharmProjects/autolens_slacs_pre_v_1/dataset/slacs_shu/"

In [148]:
agg_all = af.Aggregator(directory=str(aggregator_results_path), completed_only=True)

Aggregator loading phases... could take some time.

 A total of 419 phases and results were found.


In [149]:
phase_name = "phase[4]_mass[total]_source[fixed]/settings"

agg_shear_sie_all = agg_all.filter(agg.directory.contains(phase_name))

print("Total Outputs = ",len(agg_shear_sie_all))

Filter found a total of 41 results
Total Outputs =  41


In [150]:
datasets_shear = agg_shear_sie_all.values("dataset")
outputs_shear = agg_shear_sie_all.values("samples")
instances_shear = [samps.median_pdf_instance for samps in agg_shear_sie_all.values("samples")]
lens_no = [re.sub('slacs', '', dataset.name) for dataset in datasets_shear]

In [151]:
for lens in lens_no:
    calc_shear_err(shear_catalogue=f"{weak_data_path}{lens}_F814W_drz_sci_clean.shears",
                   outfile=f"{weak_data_path}{lens}_F814W_drz_sci_clean.shears_err")

In [152]:
data_err = fits.open(f"{weak_data_path}0822+2652_F814W_drz_sci_clean.shears_err")[1].data


print(np.mean(data_err['gamma1_err']))
print(np.mean(data_err['gamma2_err']))





9720.4437977512
73068.5573616843
